# 1.) IMPORT LIBRARIES

In [1]:
import pandas as pd
import os
import numpy as np
import re
import string

# 2.) Input DataFrame Read & Load

In [2]:
input_df = pd.read_excel('./data/Input.xlsx')
input_df.head(5)

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...


In [3]:
# page_url = "https://insights.blackcoffer.com/ai-in-healthcare-to-improve-patient-outcomes/"

In [4]:
# page_url = "https://insights.blackcoffer.com/what-if-the-creation-is-taking-over-the-creator/"

# 3.) USING BEAUTISOUP FOR EXTRACTING TEXT

In [5]:
from bs4 import BeautifulSoup
import requests

In [6]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36'}

#### => Beautisoup Extraction Functions

In [7]:
# Element Data Select 

In [8]:
def elment_selector_beauty(find_element,method,tag_name,class_name):
    if(method == 'find'):
        return find_element.find(tag_name,{'class':class_name})
    elif(method == 'find_all'):
        return find_element.find_all(tag_name,{'class':class_name})
    else:
        return None

#### => Beautisoup Topic

In [9]:
def beauty_topic_df(bs):
    beauty_topic_ul = elment_selector_beauty(bs,'find','ul','td-category')
    beauty_topic_li = elment_selector_beauty(beauty_topic_ul,'find_all','li','entry-category')
    beauty_topic = " ".join([i.get_text() for i in beauty_topic_li])
    beauty_topic = beauty_topic.replace('\n',',')
    return beauty_topic+"."

#### => Beautisoup Heading

In [10]:
def beauty_heading_df(bs):
    beauty_heading = elment_selector_beauty(bs,'find','h1','entry-title').get_text()
    return beauty_heading

#### => Beautisoup Main Content

In [11]:
def beauty_main_txt(bs):
    main_content_wrapper = elment_selector_beauty(bs,'find','div','td-main-content')
    beauty_content = elment_selector_beauty(main_content_wrapper,'find','div','td-post-content').get_text()
    return beauty_content

#### => Beautisoup Tags

In [12]:
def beauty_tags(bs):
    main_content_wrapper = elment_selector_beauty(bs,'find','div','td-main-content')
    tags_list = elment_selector_beauty(main_content_wrapper,'find','div','td-post-source-tags')
    tags_list_li = tags_list.find_all('li')
    tag_text = " ".join(i.get_text() for i in tags_list_li)
    return tag_text

#### => Mearging Functions

In [13]:
def BeautyMain(my_url):
    r = requests.get(my_url)
    soup = BeautifulSoup(r.content, 'html5lib')
    heading_txt = beauty_heading_df(soup)
    topic_txt = beauty_topic_df(soup)
    main_txt = beauty_main_txt(soup)
    tag_txt = beauty_tags(soup)
    merge_content = topic_txt+'\n'+heading_txt+'\n'+main_txt+'\n'+tag_txt
    return merge_content

# 4.) USING Selenium Data Extraction

In [14]:
from selenium.webdriver.common.by import By

In [15]:
# Element Data Select 

In [16]:
def element_list(finding,selector,identity):
    if(selector == 'tag'):
        return finding.find_elements(By.TAG_NAME,identity)
    elif(selector == 'class'):
        return finding.find_elements(By.CLASS_NAME,identity)
    else:
        return None

### => Selenium Data Extraction Functions

#### => selenium Extracting Topics

In [17]:
def SeleniumTopics(find_list):
    topics_text = "".join([j.text for i in find_list for j in element_list(i, 'class', 'tdb-category')])
    topics_text = topics_text.replace('\n',',')
    return topics_text+'.'

#### => selenium Extracting Heading

In [18]:
def SeleniumHeading(find_list):
    heading_text = "".join([j.text for i in find_list for j in element_list(i, 'class', 'tdb-title-text')])
    return heading_text

#### => selenium Extracting Content

In [19]:
def SeleniumContent(find_list):
    content_text = "".join([j.text for i in find_list for j in element_list(i, 'class', 'tdb_single_content')])
    return content_text

#### => selenium Extracting Tags

In [20]:
def SeleniumTags(find_list):
    tags_text = " ".join([k.text for i in find_list for j in element_list(i, 'class', 'tdb-tags') for k in element_list(j, 'tag', 'span')] +
                     [l.text for i in find_list for j in element_list(i, 'class', 'tdb-tags') for l in element_list(j, 'tag', 'a')])
    return tags_text

#### => selenium No Data Found / Heading Checker

In [21]:
def only_heading():
    heading1 = element_list(driver,'class','tdm-title')
    heading_text = ""
    for i in range(len(heading1)):
        text_got = heading1[i].text.lower().split(' ')
        find_str = "404,error".split(",")
        text_count = 0
        for j in find_str:
            if j in text_got:
                heading_text += heading1[i].text
                break;
    return heading_text+'\n'+"Page Not Found"

#### => selenium Page Data Extract

In [22]:
def page_content(parent_tag):
    wpb_wrapper = [j for i in parent_tag for j in element_list(i, 'class', 'wpb_wrapper')]
    # Extracting content and store in variables
    heading_data = SeleniumHeading(wpb_wrapper)
    topic_data = SeleniumTopics(wpb_wrapper)
    content_data = SeleniumContent(wpb_wrapper)
    tags_data = SeleniumTags(wpb_wrapper)
    main_content = topic_data +'\n'+ heading_data +'\n'+ content_data +'\n'+ tags_data
    return main_content

#### => selenium Main Function

In [23]:
def SeleniumMain(url):
    driver.get(url)
    page_load = driver.page_source
    # article tag from html page load
    article_tag = element_list(driver,'tag','article')
    if len(article_tag) > 0:
        return page_content(article_tag)
    else:           
        return only_heading()

# 5.) File Data Extraction

In [24]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService 
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

#### => Extracting Function

In [25]:
def Extractor(url_1):
    try:
        bm = BeautyMain(url_1)
        print("Beautisoap")
        print('.............................................................................')
        return bm
    except:
        print("Silenium")
        print('.............................................................................')
        # Selenium Driver Run
        sm = SeleniumMain(url_1)
        return sm

#### => Loop for repeating task to extract

In [26]:
for i in range(len(input_df)):
    inner_url = input_df['URL'].iloc[i]
    file_name = input_df['URL_ID'].iloc[i]
    print(inner_url)
    try:
        # In this if file exist not write again
        file_read = open(f"./data/extract-text/{file_name}.txt","r",encoding="utf-8")
        file_text = file_read.read()
        file_read.close()
        print("File Readed")
        # I didnt crete script for re-update code but if want then i have to add bottom code again / or make function for that
    except:
        # In this if file not exist then write
        print("File Not Found")
        print("Extracting From")
        extracted_data = Extractor(inner_url)
        #print(extracted_data)
        print("Extracted")
        file = open(f"./data/extract-text/{file_name}.txt","w+",encoding="utf-8")
        file.write(extracted_data)
        
        print("File Created Successfully")
    print('------------------------------------')
driver.quit()

https://insights.blackcoffer.com/ai-in-healthcare-to-improve-patient-outcomes/
File Readed
------------------------------------
https://insights.blackcoffer.com/what-if-the-creation-is-taking-over-the-creator/
File Readed
------------------------------------
https://insights.blackcoffer.com/what-jobs-will-robots-take-from-humans-in-the-future/
File Readed
------------------------------------
https://insights.blackcoffer.com/will-machine-replace-the-human-in-the-future-of-work/
File Readed
------------------------------------
https://insights.blackcoffer.com/will-ai-replace-us-or-work-with-us/
File Readed
------------------------------------
https://insights.blackcoffer.com/man-and-machines-together-machines-are-more-diligent-than-humans-blackcoffe/
File Readed
------------------------------------
https://insights.blackcoffer.com/in-future-or-in-upcoming-years-humans-and-machines-are-going-to-work-together-in-every-field-of-work/
File Readed
------------------------------------
https://

File Readed
------------------------------------
https://insights.blackcoffer.com/coronavirus-impact-on-the-hospitality-industry-2/
File Readed
------------------------------------
https://insights.blackcoffer.com/how-will-covid-19-affect-the-world-of-work-3/
File Readed
------------------------------------
https://insights.blackcoffer.com/coronavirus-impact-on-the-hospitality-industry-3/
File Readed
------------------------------------
https://insights.blackcoffer.com/estimating-the-impact-of-covid-19-on-the-world-of-work/
File Readed
------------------------------------
https://insights.blackcoffer.com/covid-19-how-have-countries-been-responding-2/
File Readed
------------------------------------
https://insights.blackcoffer.com/how-will-covid-19-affect-the-world-of-work-4/
File Readed
------------------------------------
https://insights.blackcoffer.com/lessons-from-the-past-some-key-learnings-relevant-to-the-coronavirus-crisis-2/
File Readed
------------------------------------
htt

#### => Filtering Text

In [27]:
input_files_txt = os.listdir('./data/extract-text/')

In [28]:
def removePuncuation(filename):
    file = open(f'./data/extract-text/{filename}','r+',encoding="utf-8")
    content = file.read()
    file.close()
    puncuations = list(string.punctuation)
    del puncuations[puncuations.index('.')]
    del puncuations[puncuations.index(',')]
    dels = "".join(puncuations)
    dels +="’“•‘”"
    del_puchuation = content
    del_puchuation = del_puchuation.translate(str.maketrans('', '', dels))
    return del_puchuation.strip()

In [29]:
for i in range(len(input_files_txt)):
    file = input_files_txt[i]
    file_no_ext = file.split('.txt')[0]
    print(file_no_ext)
    try:
        file_read = open(f"./data/filtered-text/{file_no_ext}-filtered.txt","r",encoding="utf-8")
        file_text = file_read.read()
        file_read.close()
        print("File Readed")
    except:
        print("File Not Found")
        print("Converting...............................")
        texture = removePuncuation(file)
        file = open(f"./data/filtered-text/{file_no_ext}-filtered.txt","w+",encoding="utf-8")
        file.write(texture)
        file.close()
        print("Converted")

100
File Readed
101
File Readed
102
File Readed
103
File Readed
104
File Readed
105
File Readed
106
File Readed
107
File Readed
108
File Readed
109
File Readed
110
File Readed
111
File Readed
112
File Readed
113
File Readed
114
File Readed
115
File Readed
116
File Readed
117
File Readed
118
File Readed
119
File Readed
120
File Readed
121
File Readed
122
File Readed
123
File Readed
124
File Readed
125
File Readed
126
File Readed
127
File Readed
128
File Readed
129
File Readed
130
File Readed
131
File Readed
132
File Readed
133
File Readed
134
File Readed
135
File Readed
136
File Readed
137
File Readed
138
File Readed
139
File Readed
140
File Readed
141
File Readed
142
File Readed
143
File Readed
144
File Readed
145
File Readed
146
File Readed
147
File Readed
148
File Readed
149
File Readed
150
File Readed
37
File Readed
38
File Readed
39
File Readed
40
File Readed
41
File Readed
42
File Readed
43
File Readed
44
File Readed
45
File Readed
46
File Readed
47
File Readed
48
File Readed
49
F

In [30]:
filter_text_files = os.listdir('./data/filtered-text/')

# 5.) OUTPUT DATAFRAME

In [31]:
import copy

In [32]:
output_df = copy.copy(input_df)
output_df.head(5)

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...


# 6.) CONTENT DATAFRAME

In [33]:
content_df = {"URL_ID":[],"Content":[]}
for i in range(len(filter_text_files)):
    file_name = filter_text_files[i]
    print(file_name)
    file_id = filter_text_files[i].split('-')[0]
    df_val = input_df[(input_df['URL_ID'] == int(file_id))]
    df_val_len = input_df[(input_df['URL_ID'] == int(file_id))].shape[0]
    
    file_read = open(f"./data/filtered-text/{file_name}","r",encoding="utf-8")
    file_text = file_read.read()
    file_read.close()
    
    if(df_val_len == 1):
        #print(file_text)
        url_id = df_val['URL_ID'].tolist()[0]
        #print(url_id)
        content_df["URL_ID"].append(url_id)
        content_df["Content"].append(file_text)
    elif (df_val_len > 1):
        print("Other")
    else:
        print(None)
    print('------------------------------------------------------')
content_df = pd.DataFrame(content_df)

100-filtered.txt
------------------------------------------------------
101-filtered.txt
------------------------------------------------------
102-filtered.txt
------------------------------------------------------
103-filtered.txt
------------------------------------------------------
104-filtered.txt
------------------------------------------------------
105-filtered.txt
------------------------------------------------------
106-filtered.txt
------------------------------------------------------
107-filtered.txt
------------------------------------------------------
108-filtered.txt
------------------------------------------------------
109-filtered.txt
------------------------------------------------------
110-filtered.txt
------------------------------------------------------
111-filtered.txt
------------------------------------------------------
112-filtered.txt
------------------------------------------------------
113-filtered.txt
-----------------------------------------------

In [34]:
content_df.head(5)

,URL_ID,Content
0,100,"Blackcoffer,What We Think,Healthcare.\nEstimat..."
1,101,Future Futurist What We Think.\nTravel and Tou...
2,102,Blackcoffer What We Think Entertainment.\nGami...
3,103,Blackcoffer What We Think Healthcare.\nWhat is...
4,104,Blackcoffer What We Think Healthcare.\nDue to ...


# 7.) Lower Text of content

In [35]:
content_df['Lower'] = content_df['Content'].apply(lambda x: x.lower())
content_df.head(5)

,URL_ID,Content,Lower
0,100,"Blackcoffer,What We Think,Healthcare.\nEstimat...","blackcoffer,what we think,healthcare.\nestimat..."
1,101,Future Futurist What We Think.\nTravel and Tou...,future futurist what we think.\ntravel and tou...
2,102,Blackcoffer What We Think Entertainment.\nGami...,blackcoffer what we think entertainment.\ngami...
3,103,Blackcoffer What We Think Healthcare.\nWhat is...,blackcoffer what we think healthcare.\nwhat is...
4,104,Blackcoffer What We Think Healthcare.\nDue to ...,blackcoffer what we think healthcare.\ndue to ...


In [36]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

# 8.) PARAGRAPH TOKENS FROM CONTENT

In [37]:
content_df['Paragraphs'] = content_df['Lower'].apply(lambda x: x.split('\n'))
content_df['Paragraphs'].head(5)

0    [blackcoffer,what we think,healthcare., estima...
1    [future futurist what we think., travel and to...
2    [blackcoffer what we think entertainment., gam...
3    [blackcoffer what we think healthcare., what i...
4    [blackcoffer what we think healthcare., due to...
Name: Paragraphs, dtype: object

<h3 style="color: red">=> Getting Paragraph length and store in output_df</h3>

In [38]:
output_df['Paragraph Length'] = content_df['Paragraphs'].apply(lambda x:len(x))
output_df.head(5)

,URL_ID,URL,Paragraph Length
0,37,https://insights.blackcoffer.com/ai-in-healthc...,99
1,38,https://insights.blackcoffer.com/what-if-the-c...,37
2,39,https://insights.blackcoffer.com/what-jobs-wil...,69
3,40,https://insights.blackcoffer.com/will-machine-...,101
4,41,https://insights.blackcoffer.com/will-ai-repla...,29


# 9.) SENTENCE TOKENS FROM CONTENT

In [39]:
content_df['Sentence'] = content_df['Lower'].apply(lambda x: sent_tokenize(x))
content_df.head(5)

,URL_ID,Content,Lower,Paragraphs,Sentence
0,100,"Blackcoffer,What We Think,Healthcare.\nEstimat...","blackcoffer,what we think,healthcare.\nestimat...","[blackcoffer,what we think,healthcare., estima...","[blackcoffer,what we think,healthcare., estima..."
1,101,Future Futurist What We Think.\nTravel and Tou...,future futurist what we think.\ntravel and tou...,"[future futurist what we think., travel and to...","[future futurist what we think., travel and to..."
2,102,Blackcoffer What We Think Entertainment.\nGami...,blackcoffer what we think entertainment.\ngami...,"[blackcoffer what we think entertainment., gam...","[blackcoffer what we think entertainment., gam..."
3,103,Blackcoffer What We Think Healthcare.\nWhat is...,blackcoffer what we think healthcare.\nwhat is...,"[blackcoffer what we think healthcare., what i...","[blackcoffer what we think healthcare., what i..."
4,104,Blackcoffer What We Think Healthcare.\nDue to ...,blackcoffer what we think healthcare.\ndue to ...,"[blackcoffer what we think healthcare., due to...","[blackcoffer what we think healthcare., due to..."


In [40]:
output_df['Sentence Length'] = content_df['Sentence'].apply(lambda x:len(x))
output_df.head(5)

,URL_ID,URL,Paragraph Length,Sentence Length
0,37,https://insights.blackcoffer.com/ai-in-healthc...,99,53
1,38,https://insights.blackcoffer.com/what-if-the-c...,37,9
2,39,https://insights.blackcoffer.com/what-jobs-wil...,69,57
3,40,https://insights.blackcoffer.com/will-machine-...,101,31
4,41,https://insights.blackcoffer.com/will-ai-repla...,29,48


# 10.) WORD TOKENS FROM CONTENT

In [41]:
content_df['Tokens'] = content_df['Lower'].apply(lambda x: word_tokenize(x))
content_df.head(5)

,URL_ID,Content,Lower,Paragraphs,Sentence,Tokens
0,100,"Blackcoffer,What We Think,Healthcare.\nEstimat...","blackcoffer,what we think,healthcare.\nestimat...","[blackcoffer,what we think,healthcare., estima...","[blackcoffer,what we think,healthcare., estima...","[blackcoffer, ,, what, we, think, ,, healthcar..."
1,101,Future Futurist What We Think.\nTravel and Tou...,future futurist what we think.\ntravel and tou...,"[future futurist what we think., travel and to...","[future futurist what we think., travel and to...","[future, futurist, what, we, think, ., travel,..."
2,102,Blackcoffer What We Think Entertainment.\nGami...,blackcoffer what we think entertainment.\ngami...,"[blackcoffer what we think entertainment., gam...","[blackcoffer what we think entertainment., gam...","[blackcoffer, what, we, think, entertainment, ..."
3,103,Blackcoffer What We Think Healthcare.\nWhat is...,blackcoffer what we think healthcare.\nwhat is...,"[blackcoffer what we think healthcare., what i...","[blackcoffer what we think healthcare., what i...","[blackcoffer, what, we, think, healthcare, ., ..."
4,104,Blackcoffer What We Think Healthcare.\nDue to ...,blackcoffer what we think healthcare.\ndue to ...,"[blackcoffer what we think healthcare., due to...","[blackcoffer what we think healthcare., due to...","[blackcoffer, what, we, think, healthcare, ., ..."


<h3 style="color: red">=> Apply Lemmetisation On Tokens</h3>

In [42]:
random1 = content_df['Tokens'][0]
random1

['blackcoffer',
 ',',
 'what',
 'we',
 'think',
 ',',
 'healthcare',
 '.',
 'estimating',
 'the',
 'impact',
 'of',
 'covid19',
 'on',
 'the',
 'world',
 'of',
 'work',
 'will',
 'covid19',
 'end',
 'globalization',
 'globalization',
 'globalization',
 'envisages',
 'a',
 'borderless',
 'world',
 'or',
 'seeks',
 'the',
 'world',
 'as',
 'a',
 'global',
 'village',
 'it',
 'is',
 'attributed',
 'to',
 'the',
 'accelerated',
 'flow',
 'of',
 'goods',
 ',',
 'people',
 ',',
 'capital',
 ',',
 'information',
 ',',
 'and',
 'energy',
 'across',
 'borders',
 ',',
 'often',
 'enabled',
 'by',
 'technological',
 'developments',
 '.',
 'globalization',
 'trends',
 'starting',
 'from',
 '1990s',
 'globalization',
 'dominated',
 'worlds',
 'economic',
 'order',
 '.',
 'antiglobalization',
 'wave',
 'globalization',
 'had',
 'already',
 'begun',
 'to',
 'stagnate',
 'since',
 'the',
 '200809',
 'global',
 'financial',
 'crisis',
 '.',
 'slowing',
 'down',
 'of',
 'trade',
 'trade',
 'as',
 'a',
 

In [54]:
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
ps = PorterStemmer()
wl = WordNetLemmatizer()

In [95]:
def stemmer(obj):
    ignoring = list(',.-_')
    return [wl.lemmatize(word,pos="v") for word in obj if word not in ignoring]

In [96]:
checker_1  = stemmer(random1)
checker_1

['blackcoffer',
 'what',
 'we',
 'think',
 'healthcare',
 'estimate',
 'the',
 'impact',
 'of',
 'covid19',
 'on',
 'the',
 'world',
 'of',
 'work',
 'will',
 'covid19',
 'end',
 'globalization',
 'globalization',
 'globalization',
 'envisage',
 'a',
 'borderless',
 'world',
 'or',
 'seek',
 'the',
 'world',
 'as',
 'a',
 'global',
 'village',
 'it',
 'be',
 'attribute',
 'to',
 'the',
 'accelerate',
 'flow',
 'of',
 'goods',
 'people',
 'capital',
 'information',
 'and',
 'energy',
 'across',
 'border',
 'often',
 'enable',
 'by',
 'technological',
 'developments',
 'globalization',
 'trend',
 'start',
 'from',
 '1990s',
 'globalization',
 'dominate',
 'worlds',
 'economic',
 'order',
 'antiglobalization',
 'wave',
 'globalization',
 'have',
 'already',
 'begin',
 'to',
 'stagnate',
 'since',
 'the',
 '200809',
 'global',
 'financial',
 'crisis',
 'slow',
 'down',
 'of',
 'trade',
 'trade',
 'as',
 'a',
 'percentage',
 'of',
 'global',
 'gdp',
 'increase',
 'from',
 '39',
 'in',
 '199

In [92]:
wl.lemmatize('investigated',pos="v")

'investigate'